# PC-AI Comprehensive LLM Evaluation Suite

This notebook drives a parameter sweep across PC-AI compiled backends, captures system resource telemetry, and graphs performance/quality metrics. It also includes optional hooks for standard LLM benchmarks via `lm-evaluation-harness`.


In [ ]:
import os, json, time, datetime, subprocess, sys, socket
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request

PROJECT_ROOT = Path(r'C:\\Users\\david\\PC_AI')
REPORTS_DIR = PROJECT_ROOT / 'Reports' / 'llm-eval' / datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
REPORTS_DIR.mkdir(parents=True, exist_ok=True)
print('Reports:', REPORTS_DIR)

def find_powershell():
    for exe in ['pwsh', 'powershell']:
        try:
            subprocess.run([exe, '-NoProfile', '-Command', '$PSVersionTable.PSVersion'], check=True, capture_output=True, text=True)
            return exe
        except Exception:
            continue
    raise RuntimeError('PowerShell not found')

POWERSHELL = find_powershell()
print('Using PowerShell:', POWERSHELL)

def run_ps(cmd, check=True):
    result = subprocess.run([POWERSHELL, '-NoProfile', '-ExecutionPolicy', 'Bypass', '-Command', cmd],
                            capture_output=True, text=True)
    if check and result.returncode != 0:
        raise RuntimeError(f'PowerShell failed: {result.stderr}\nSTDOUT:\n{result.stdout}')
    return result

def get_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]


## Load PC-AI modules
This ensures the evaluation and performance modules are available.


In [ ]:
ps = f'''
Import-Module '{PROJECT_ROOT / 'Modules' / 'PcaiInference.psd1'}' -Force -ErrorAction SilentlyContinue
Import-Module '{PROJECT_ROOT / 'Modules' / 'PC-AI.Evaluation' / 'PC-AI.Evaluation.psd1'}' -Force
Import-Module '{PROJECT_ROOT / 'Modules' / 'PC-AI.Performance' / 'PC-AI.Performance.psd1'}' -Force
Get-PcaiInferenceStatus | Format-List
'''
print(run_ps(ps).stdout)


## Model + binary discovery
We auto-detect a test model and compiled backend binaries. For llama.cpp we need a `.gguf`. For mistral.rs, a SafeTensors model is acceptable. Set `PCAI_TEST_MODEL` to override.


In [ ]:
def find_model_path():
    env_path = os.environ.get('PCAI_TEST_MODEL')
    if env_path and Path(env_path).exists():
        return Path(env_path)

    models_dir = PROJECT_ROOT / 'Models'
    gguf = list(models_dir.rglob('*.gguf'))
    if gguf:
        return gguf[0]

    gemma = models_dir / 'functiongemma-270m-it' / 'model.safetensors'
    if gemma.exists():
        return gemma

    lm = Path(os.environ.get('LOCALAPPDATA','')) / 'lm-studio' / 'models'
    if lm.exists():
        candidates = list(lm.rglob('*.gguf'))
        if candidates:
            return candidates[0]

    ollama = Path.home() / '.ollama' / 'models' / 'blobs'
    if ollama.exists():
        candidates = list(ollama.rglob('*.gguf'))
        if candidates:
            return candidates[0]

    return None

def find_binary(name):
    candidates = [
        Path(os.environ.get('PCAI_BIN_DIR','')) if os.environ.get('PCAI_BIN_DIR') else None,
        Path(os.environ.get('PCAI_LOCAL_BIN','')) if os.environ.get('PCAI_LOCAL_BIN') else None,
        Path.home() / '.local' / 'bin',
        Path(os.environ.get('CARGO_TARGET_DIR','')) / 'release' if os.environ.get('CARGO_TARGET_DIR') else None,
        Path('T:/RustCache/cargo-target/release')
    ]
    for base in [c for c in candidates if c]:
        candidate = base / name
        if candidate.exists():
            return candidate
    return None

MODEL_PATH = find_model_path()
LLAMACPP_BIN = find_binary('pcai-llamacpp.exe')
MISTRAL_BIN = find_binary('pcai-mistralrs.exe')

print('MODEL_PATH:', MODEL_PATH)
print('LLAMACPP_BIN:', LLAMACPP_BIN)
print('MISTRAL_BIN:', MISTRAL_BIN)

if MODEL_PATH is None or LLAMACPP_BIN is None:
    raise RuntimeError('Missing gguf model or pcai-llamacpp.exe. Set PCAI_TEST_MODEL and ensure binaries are in .local/bin.')


## Parameter grid
Tight defaults to keep runtime reasonable; expand for deeper sweeps.


In [ ]:
temperatures = [0.2, 0.8]
max_tokens = [64, 128]
gpu_layers = [-1, 0]
datasets = ['diagnostic', 'safety']
max_test_cases = 3

backends = ['llamacpp-bin']
if os.environ.get('PCAI_INCLUDE_MISTRALRS') == '1' and MISTRAL_BIN:
    backends.append('mistralrs-bin')

print('Backends:', backends)
print('Datasets:', datasets)


## Server control helpers
We start a compiled server once per backend and GPU layer setting, then run all parameter sweeps against it (HTTP mode).


In [ ]:
def build_config(backend: str, model_path: Path, base_url: str, gpu_layers: int = -1):
    resolved_model = model_path
    if str(model_path).lower().endswith('.safetensors'):
        resolved_model = model_path.parent

    host, port = base_url.replace('http://', '').split(':')
    config = {
        'backend': { 'type': backend },
        'model': {
            'path': str(resolved_model),
            'generation': {
                'max_tokens': 512,
                'temperature': 0.7,
                'top_p': 0.95,
                'stop': []
            }
        },
        'server': { 'host': host, 'port': int(port), 'cors': True }
    }
    if backend == 'llama_cpp' and gpu_layers >= 0:
        config['backend']['n_gpu_layers'] = gpu_layers
    return config

def wait_health(base_url: str, timeout_s: int = 60):
    deadline = time.time() + timeout_s
    while time.time() < deadline:
        try:
            with urllib.request.urlopen(base_url + '/health', timeout=3) as resp:
                if resp.status == 200:
                    return True
        except Exception:
            time.sleep(1)
    return False

def start_server(binary: Path, config_path: Path):
    env = os.environ.copy()
    env['PCAI_CONFIG'] = str(config_path)
    return subprocess.Popen([str(binary)], cwd=binary.parent, env=env)


## System resource monitoring helper


In [ ]:
def start_resource_monitor(output_csv: Path, duration_s: int, refresh_s: int = 1):
    ps = f'''
Import-Module '{PROJECT_ROOT / 'Modules' / 'PC-AI.Performance' / 'PC-AI.Performance.psd1'}' -Force
Watch-SystemResources -Duration {duration_s} -RefreshInterval {refresh_s} -OutputMode Object | Export-Csv -NoTypeInformation -Path '{output_csv}'
'''
    return subprocess.Popen([POWERSHELL, '-NoProfile', '-ExecutionPolicy', 'Bypass', '-Command', ps],
                            stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)


## Run evaluation grid
Uses the PC-AI evaluation harness in HTTP mode against the compiled server.


In [ ]:
results = []

for backend in backends:
    binary = MISTRAL_BIN if backend == 'mistralrs-bin' else LLAMACPP_BIN
    backend_type = 'mistral_rs' if backend == 'mistralrs-bin' else 'llama_cpp'

    for server_gpu in gpu_layers:
        port = get_free_port()
        base_url = f'http://127.0.0.1:{port}'
        config = build_config(backend_type, MODEL_PATH, base_url, gpu_layers=server_gpu)
        config_path = REPORTS_DIR / f'config_{backend_type}_gpu{server_gpu}.json'
        config_path.write_text(json.dumps(config, indent=2))

        proc = start_server(binary, config_path)
        if not wait_health(base_url, timeout_s=120):
            proc.kill()
            raise RuntimeError(f'Backend {backend} failed to start')

        try:
            for dataset in datasets:
                for temp in temperatures:
                    for max_tok in max_tokens:
                        run_id = f'{backend}_{dataset}_t{temp}_mt{max_tok}_gpu{server_gpu}'.replace('.', 'p')
                        out_json = REPORTS_DIR / f'{run_id}.json'
                        out_csv = REPORTS_DIR / f'{run_id}_resources.csv'

                        duration = max(20, max_test_cases * 5)
                        monitor = start_resource_monitor(out_csv, duration_s=duration, refresh_s=1)

                        cmd = f"""& '{PROJECT_ROOT / 'Tests' / 'Evaluation' / 'Invoke-InferenceEvaluation.ps1'}' `
    -Backend http `
    -BaseUrl {base_url} `
    -ModelPath '{MODEL_PATH}' `
    -Dataset {dataset} `
    -MaxTokens {max_tok} `
    -Temperature {temp} `
    -GpuLayers {server_gpu} `
    -MaxTestCases {max_test_cases} `
    -OutputPath '{out_json}'
"""
                        print('Running:', run_id)
                        ps_result = run_ps(cmd, check=False)

                        try:
                            monitor.wait(timeout=duration + 10)
                        except subprocess.TimeoutExpired:
                            monitor.kill()

                        if ps_result.returncode != 0:
                            print('Run failed:', run_id, ps_result.stderr)
                            continue

                        if not out_json.exists():
                            print('Missing output:', out_json)
                            continue

                        payload = json.loads(out_json.read_text())
                        summary = None
                        for item in payload.get('Results', []):
                            if item.get('Key') == 'http':
                                summary = item.get('Value')
                                break
                        if summary is None:
                            summary = {}

                        results.append({
                            'run_id': run_id,
                            'backend': backend,
                            'dataset': dataset,
                            'temperature': temp,
                            'max_tokens': max_tok,
                            'gpu_layers': server_gpu,
                            'pass_rate': summary.get('PassRate'),
                            'avg_score': summary.get('AverageScore'),
                            'avg_latency_ms': summary.get('AverageLatency'),
                            'resources_csv': str(out_csv)
                        })
        finally:
            proc.terminate()
            try:
                proc.wait(timeout=10)
            except subprocess.TimeoutExpired:
                proc.kill()

df = pd.DataFrame(results)
display(df.head())
df.to_csv(REPORTS_DIR / 'summary.csv', index=False)
print('Saved summary:', REPORTS_DIR / 'summary.csv')


## Plot performance + quality metrics


In [ ]:
if df.empty:
    raise RuntimeError('No results collected. Check logs in Reports for failures.')

plt.figure(figsize=(12, 5))
for backend in df['backend'].unique():
    subset = df[df['backend'] == backend]
    plt.plot(subset['avg_latency_ms'], label=f'{backend} latency')
plt.title('Average Latency per Run')
plt.xlabel('Run Index')
plt.ylabel('Latency (ms)')
plt.legend()
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 5))
for backend in df['backend'].unique():
    subset = df[df['backend'] == backend]
    plt.plot(subset['avg_score'], label=f'{backend} score')
plt.title('Average Quality Score per Run')
plt.xlabel('Run Index')
plt.ylabel('Score')
plt.legend()
plt.tight_layout()
plt.show()


## Plot system resource telemetry (CPU + memory)


In [ ]:
def plot_resources(csv_path):
    data = pd.read_csv(csv_path)
    if data.empty:
        return
    fig, ax1 = plt.subplots(figsize=(12, 5))
    ax1.plot(data['CpuPercent'], label='CPU %')
    ax1.set_ylabel('CPU %')
    ax2 = ax1.twinx()
    ax2.plot(data['MemoryPercent'], color='orange', label='Memory %')
    ax2.set_ylabel('Memory %')
    ax1.set_title(Path(csv_path).name)
    fig.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

for csv_path in df['resources_csv'].head(3):
    plot_resources(csv_path)


## Optional: Standard LLM benchmark suite (lm-evaluation-harness)
This step uses `lm-evaluation-harness` (EleutherAI) to run short subsets of common tasks (ARC Easy, HellaSwag, TruthfulQA). It is optional and will skip if installation fails.


In [ ]:
def ensure_lm_eval():
    try:
        import lm_eval  # noqa: F401
        return True
    except Exception:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'lm-eval[openai]>=0.4.4'])
            return True
        except Exception as e:
            print('lm-eval install failed:', e)
            return False

if ensure_lm_eval():
    tasks = 'arc_easy,hellaswag,truthfulqa_mc'
    base_url = 'http://127.0.0.1:8080/v1'
    cmd = [
        sys.executable, '-m', 'lm_eval',
        '--model', 'openai-chat-completions',
        '--model_args', f'model=pcai-inference,api_base={base_url},api_key=sk-local',
        '--tasks', tasks,
        '--limit', '5',
        '--output_path', str(REPORTS_DIR / 'lm_eval_results.json')
    ]
    print('Running lm-eval:', ' '.join(cmd))
    subprocess.run(cmd, check=False)
else:
    print('lm-eval not available; skipping standard benchmark suite.')


## Optimization recommendation
We compute a simple composite score to pick a high-performance configuration.


In [ ]:
df_clean = df.dropna(subset=['avg_score', 'avg_latency_ms']).copy()
df_clean['score_norm'] = df_clean['avg_score']
df_clean['lat_norm'] = 1.0 - (df_clean['avg_latency_ms'] / df_clean['avg_latency_ms'].max())
df_clean['composite'] = (df_clean['score_norm'] * 0.6) + (df_clean['lat_norm'] * 0.4)
best = df_clean.sort_values('composite', ascending=False).head(5)
display(best)
